<a href="https://colab.research.google.com/github/JMartinArocha/MasterBigData/blob/main/Algoritmos_evolutivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import random
import numpy as np

distancias = np.array([
    [0, 19.6, 16, 28.3, 19.5],
    [19.6, 0, 20.7, 45, 32.6],
    [16, 20.7, 0, 41.4, 20.2],
    [28.3, 45, 41.4, 0, 30.5],
    [19.5, 32.6, 20.2, 30.5, 0]
])

mejor_gen = 0
mejor_fitness = 0

# Función para calcular la distancia total de una ruta
def fitness(ruta):
    return sum(distancias[ruta[i], ruta[i + 1]] for i in range(len(ruta) - 1))

# Función de mutación que evita duplicados
def mutation(individuo):
    c1, c2 = random.sample(range(len(individuo)), 2)
    individuo[c1], individuo[c2] = individuo[c2], individuo[c1]
    return individuo

# Función de recombinación que asegura rutas válidas
def recombination(individuo1, individuo2):
    punto = random.randint(1, len(individuo1) - 1)
    nueva_ruta1 = individuo1[:punto] + [ciudad for ciudad in individuo2 if ciudad not in individuo1[:punto]]
    nueva_ruta2 = individuo2[:punto] + [ciudad for ciudad in individuo1 if ciudad not in individuo2[:punto]]
    # Aseguramos que no haya duplicados
    nueva_ruta1 = list(dict.fromkeys(nueva_ruta1))
    nueva_ruta2 = list(dict.fromkeys(nueva_ruta2))
    return nueva_ruta1, nueva_ruta2


def seleccionar_individuos(poblacion, num_seleccionados):
    seleccionados = []
    while len(seleccionados) < num_seleccionados:
        if len(poblacion) > 1:
            individuo1, individuo2 = random.sample(poblacion, 2)
            seleccionado = min([individuo1, individuo2], key=fitness)
        else:
            seleccionado = poblacion[0]
        seleccionados.append(seleccionado)
    return seleccionados


def algoritmo_evolutivo(poblacion_inicial, prob_mutacion, prob_recombinacion, generaciones, elitismo=True):
    poblacion = poblacion_inicial
    print("Población Inicial: ")
    print(poblacion, "\n")

    # Inicializar el mejor individuo global con un individuo de la población inicial
    mejor_individuo_global = poblacion_inicial[0].copy()
    mejor_fitness_global = fitness(mejor_individuo_global)

    for gen in range(generaciones):
        # Calcular fitness para toda la población
        fitness_poblacion = [fitness(individuo) for individuo in poblacion]


       # Identificar y conservar el mejor individuo
        if elitismo and mejor_individuo_global:
            # poblacion[-1] = mejor_individuo_global
            mejor_individuo = min(poblacion, key=fitness)
            mejor_fitness = fitness(mejor_individuo)
            if mejor_fitness < mejor_fitness_global:
                mejor_individuo_global = mejor_individuo.copy()
                mejor_fitness_global = mejor_fitness

        # Seleccionar individuos para reproducción
        seleccionados = seleccionar_individuos(poblacion, len(poblacion) // 2)

        # Crear nuevos individuos
        nuevos_individuos = []
        while len(nuevos_individuos) < len(poblacion) - 1:
            if random.random() < prob_recombinacion and len(seleccionados) > 1:
                padre1, padre2 = random.sample(seleccionados, 2)
                hijo1, hijo2 = recombination(padre1, padre2)
                nuevos_individuos.extend([mutation(hijo1), mutation(hijo2)])
            else:
                individuo = random.choice(seleccionados)
                nuevos_individuos.append(mutation(individuo))

        # Incluir el mejor individuo global en la nueva población
        if elitismo:
            nuevos_individuos.append(mejor_individuo_global)

        poblacion = nuevos_individuos[:len(poblacion)]

        # Mostrar progreso
        if gen % 10 == 0:
          mejor_actual = min(poblacion, key=fitness)
          print(f"Generación {gen}: Mejor Ruta = {mejor_actual}, Fitness = {fitness(mejor_actual)}")
        # print(f"Generación {gen}: Mejor Ruta = {mejor_individuo_global}, Fitness = {mejor_fitness_global}")


    return mejor_individuo_global, mejor_fitness_global



# Hiperparámetros
poblacion_inicial = [list(np.random.permutation([0, 1, 2, 3, 4])) for _ in range(5)]
prob_recombinacion = 0.7 # probabilidad de recombinación del 70%
prob_mutacion = 0.1 # probabilidad de mutación del 10%
generaciones = 100

# Ejecución del algoritmo
mejor_individuo, mejor_fitness = algoritmo_evolutivo(poblacion_inicial, prob_mutacion, prob_recombinacion, generaciones)
print('-------------')
print("Mejor individuo encontrado:", mejor_individuo)
print("Fitness del mejor individuo:", mejor_fitness)


Población Inicial: 
[[2, 0, 3, 1, 4], [3, 0, 1, 4, 2], [1, 3, 2, 0, 4], [3, 0, 4, 1, 2], [3, 4, 1, 2, 0]] 

Generación 0: Mejor Ruta = [3, 4, 0, 2, 1], Fitness = 86.7
Generación 10: Mejor Ruta = [1, 0, 4, 2, 3], Fitness = 100.69999999999999
Generación 20: Mejor Ruta = [3, 1, 0, 4, 2], Fitness = 104.3
Generación 30: Mejor Ruta = [3, 4, 1, 0, 2], Fitness = 98.7
Generación 40: Mejor Ruta = [2, 0, 3, 4, 1], Fitness = 107.4
Generación 50: Mejor Ruta = [3, 0, 1, 2, 4], Fitness = 88.80000000000001
Generación 60: Mejor Ruta = [1, 0, 2, 4, 3], Fitness = 86.3
Generación 70: Mejor Ruta = [3, 4, 0, 2, 1], Fitness = 86.7
Generación 80: Mejor Ruta = [2, 4, 3, 0, 1], Fitness = 98.6
Generación 90: Mejor Ruta = [3, 2, 1, 0, 4], Fitness = 101.19999999999999
-------------
Mejor individuo encontrado: [3, 2, 4, 0, 1]
Fitness del mejor individuo: 86.3
